### FKP Calculation

In [3]:
import os
import glob
import fitsio
import argparse
import numpy as np
from astropy.io import fits
from astropy.table import Table
from matplotlib import pyplot as plt

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['legend.fontsize'] = 14
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [7]:
nran_list = {'ELG_LOPnotqso':10, 'LRG':8, 'QSO':2}

survey  ='Y1'
specver ='iron'
mockver ='v4_2'
tracer  = 'ELG_LOPnotqso'
region  ='NGC' #region NGC, SGC, GCcomb
dn = 'dat'
nran = nran_list[tracer]

MOCKNUM = 0
file_path = f'/pscratch/sd/s/shengyu/mocks/{survey}/Abacus_{mockver}/altmtl{MOCKNUM}/{specver}/mock{MOCKNUM}/LSScats'
mock_type = 'clustering' #clustering or full_HPmapcut (before z selection and FKP weight assigments)
catalog_fn = glob.glob(os.path.join(file_path, f'{tracer}_{region}_{mock_type}.{dn}.fits'))[0]

In [ ]:
c = 299792 # speed of light in km/s
P0_values = {'QSO': 6000, 'LRG': 10000, 'ELG_LOPnotqso': 4000, 'BGS': 7000}

def catasfun(file_path, catas_type):
    P0 = P0_values.get(tracer, None)
    catalog_fn = file_path+ f'/{tracer}_{region}_{mock_type}.{dn}.fits'
    catalog=Table(fitsio.read(catalog_fn))
    try:
        catalog[f'Z_{catas_type}']
    except KeyError:
        raise ValueError(f"Invalid Zcatas type: '{catas_type}'.")
    if f'FKP_{catas_type}' in catalog.colnames:
        print(f'{catas_type} catastrophics FKP ready:', catalog_fn)
    else:
        import time
        T0=time.time()
        catalog[f'FKP_{catas_type}'] = catalog['WEIGHT_FKP']*1
        # change WEIGHT_FKP of catastrophics
        nz        = np.loadtxt(file_path+f'/QSO_{region}_nz_{catas_type}.txt')
        ## find the nz of the true redshift
        ind_rawNZ = np.argmin(abs(catalog['Z']-nz[:,0][:,np.newaxis]),axis=0)
        ## caluclate the completeness rescaling of nz for FKP weight
        norm = catalog['NX']/nz[ind_rawNZ,3]
        dv   = (catalog[f'Z_{catas_type}']-catalog['Z'])/(1+catalog['Z'])*c
        dz   = (catalog[f'Z_{catas_type}']-catalog['Z'])
        # note that FKP changes are fewer than z changes, this is because
        ## 1. 1% of the extra catas has dv<1000km/s for z=1.32 catas(negligible)
        ## 2. 99% of the extra catas was not from 0.8<Z_RAW<1.6=> need to find the norm of the closest NX for FKP calculation as they had norm==0
        tmp      = np.argsort(catalog,order=['RA', 'DEC'])
        catalog  = catalog[tmp]
        norm     = norm[tmp]
        dv       = dv[tmp]
        NX       = catalog['NX']*1
        norm[norm==0] = np.nan
        print('there are {} samples to find new FKP'.format(np.sum((dv!=0)&(np.isnan(norm)))))
        for ID in np.where((dv!=0)&(np.isnan(norm)))[0]:
            if (2<ID)&(ID<len(catalog)-2):
                norm[ID] = np.nanmedian(norm[[ID-2,ID-1,ID+1,ID+2]])
            elif ID<2:
                norm[ID] = np.nanmedian(norm[[ID+1,ID+2]])
            elif ID>len(catalog)-2:
                norm[ID] = np.nanmedian(norm[[ID-2,ID-1]])
            # update NX for norm ==0
            ind_newNZ = np.argmin(abs(catalog[f'Z_{catas_type}'][ID]-nz[:,0]))
            NX[ID] = norm[ID]*nz[ind_newNZ,3]
        #select all catastrophics
        sel = dv!=0
        ind_newNZ = np.argmin(abs(catalog[f'Z_{catas_type}'][sel]-nz[:,0][:,np.newaxis]),axis=0)
        # update NX and WEIGHT_FKP columns for all catastrophics
        NX[sel]  = norm[sel]*nz[ind_newNZ,3]
        catalog[f'FKP_{catas_type}'][sel] = 1 / (NX[sel]*P0+1)
        catalog[f'FKP_{catas_type}'][np.isnan(catalog[f'FKP_{catas_type}'])] = 1
        # find the nz of the catastrophics redshift
        print('implement {} catastrophophics took time: {:.2f}s'.format(catas_type, time.time()-T0))
        catalog.write(catalog_fn,overwrite=True)
        print(f'{catas_type} catastrophics FKP corrected')

In [14]:
for catas_type in ['realistic','failures']:
    catasfun(file_path, catas_type=catas_type)


/pscratch/sd/s/shengyu/mocks/Y1/Abacus_v4_2/altmtl0/iron/mock0/LSScats
/pscratch/sd/s/shengyu/mocks/Y1/Abacus_v4_2/altmtl0/iron/mock0/LSScats/ELG_LOPnotqso_NGC_clustering.dat.fits
there are 255 samples to find new FKP
implement realistic catastrophophics took time: 4.68s
realistic catastrophics FKP corrected
/pscratch/sd/s/shengyu/mocks/Y1/Abacus_v4_2/altmtl0/iron/mock0/LSScats
/pscratch/sd/s/shengyu/mocks/Y1/Abacus_v4_2/altmtl0/iron/mock0/LSScats/ELG_LOPnotqso_NGC_clustering.dat.fits
there are 744 samples to find new FKP
implement failures catastrophophics took time: 4.80s
failures catastrophics FKP corrected


In [41]:
catalog=Table(fitsio.read(catalog_fn))
print(catalog.colnames)
print(np.nonzero(catalog['WEIGHT_FKP']-catalog['FKP_realistic']))

['TARGETID', 'RA', 'DEC', 'Z', 'NTILE', 'PHOTSYS', 'FRAC_TLOBS_TILES', 'Z_realistic', 'Z_failures', 'WEIGHT', 'WEIGHT_ZFAIL', 'WEIGHT_COMP', 'WEIGHT_SYS', 'NX', 'WEIGHT_FKP', 'FKP_realistic', 'FKP_failures']
(array([    184,     511,     548, ..., 1812590, 1812710, 1812785]),)
